In [ ]:
pip install "crewai[tools]"

In [ ]:
pip install langchain-google-genai

In [ ]:
pip install --upgrade crewai crewai_tools langchain-google-genai

In [ ]:
pip install litellm

In [ ]:
pip install "crewai[google-genai]"

In [ ]:
import os
import pandas as pd
import logging # For secure error logging
from dotenv import load_dotenv # For loading secrets
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool
from crewai.tools import tool
from crewai.llm import LLM

# --- Configuration ---
# This line loads all variables from your .env file
load_dotenv()

# Check if the keys are actually loaded.
if "GOOGLE_API_KEY" not in os.environ:
    raise EnvironmentError("GOOGLE_API_KEY not found in .env file")
if "SERPER_API_KEY" not in os.environ:
    raise EnvironmentError("SERPER_API_KEY not found in .env file")

# Instantiate the web search tool
search_tool = SerperDevTool()

# --- LLM now securely reads from the environment ---
gemini_llm = LLM(
    model="gemini/gemini-1.5-pro-latest",
    config={
        # This securely reads the key loaded by load_dotenv()
        "api_key": os.environ["GOOGLE_API_KEY"] 
    }
)

# --- Load file configuration from .env ---
DATA_DIR = os.environ.get("DATA_DIR", ".")
REFINING_FILE = os.environ.get("REFINING_FILE", "PET_SUM_SNDW_DCUS_NUS_4.xls - Data 2.csv")


# --- Helper function for cleaning EIA dates ---
def clean_eia_date(date_series):
    """Converts EIA's Excel serial dates to datetime objects."""
    return pd.to_datetime(date_series, unit='D', origin='1899-12-30')

# --- 2. Define Your Custom Data Tools ---

@tool("refining_analyzer_tool")
def refining_analyzer_tool(query: str) -> str:
    """
    Analyzes U.S. refinery operations.
    Use this tool to get the latest refinery utilization percentage and crude oil input.
    The query is ignored, it always returns the latest data.
    """
    # Read file path from config
    file_path = os.path.join(DATA_DIR, REFINING_FILE)
    
    try:
        # Check if file exists before trying to read it
        if not os.path.exists(file_path):
            logging.error(f"File not found at path: {file_path}")
            return "Error: Data file not found. Please check configuration."

        df = pd.read_csv(file_path, skiprows=2)
        df.columns = df.columns.str.strip()
        df['Date'] = clean_eia_date(df['Date'])

        util_col = '4-Week Avg U.S. Percent Utilization of Refinery Operable Capacity (Percent)'
        input_col = '4-Week Avg U.S. Refiner Net Input of Crude Oil  (Thousand Barrels per Day)'
        df[util_col] = pd.to_numeric(df[util_col], errors='coerce')
        df[input_col] = pd.to_numeric(df[input_col], errors='coerce')

        latest_data = df.sort_values(by='Date', ascending=False).iloc[0]

        result = (
            f"Latest Refinery Analysis:\n"
            f"Date: {latest_data['Date'].strftime('%Y-%m-%d')}\n"
            f"Utilization: {latest_data[util_col]:.1f}%\n"
            f"Net Input: {latest_data[input_col]:,.0f} Thousand Barrels per Day"
        )
        return result
    
    # Secure Error Handling
    except Exception as e:
        # Log the full error for you
        logging.error(f"Error in refining_analyzer_tool: {e}", exc_info=True)
        # Return a generic error to the AI
        return "Error: An internal error occurred while analyzing refining data."

@tool("market_demand_tool")
def market_demand_tool(market_name: str) -> str:
    """
    Analyzes demand for a specific market (e.g., 'Texas' or 'New York').
    Returns the latest 'Product Supplied' (demand) and 'Natural Gas Price' for that market.
    """
    # TODO: Harden this tool
    return f"Mock Data: {market_name} shows high demand. (Product Supplied: 9,000; NG Price: $3.50)"

@tool("logistics_volume_tool")
def logistics_volume_tool(query: str) -> str:
    """
    Analyzes U.S. import and export volumes.
    The query is ignored, it returns the latest data.
    """
    # TODO: Harden this tool
    return "Mock Data: Imports are at 6,500 and Exports are at 4,000 (Thousand Barrels per Day)."

@tool("market_benchmark_tool")
def market_benchmark_tool(query: str) -> str:
    """
    Gets the latest Henry Hub Natural Gas spot price.
    The query is ignored.
    """
    # TODO: Harden this tool
    return "Mock Data: Latest Henry Hub Spot Price: $2.95 on 2025-10-20"

@tool("midstream_margin_tool")
def midstream_margin_tool(query: str) -> str:
    """
    Analyzes the 'midstream margin' by comparing Wellhead and Citygate prices.
    The query is ignored.
    """
    # TODO: Harden this tool
    return "Mock Data: Latest Midstream Margin: Wellhead: $2.10, Citygate: $3.80. Spread: $1.70"

@tool("storage_inventory_tool")
def storage_inventory_tool(query: str) -> str:
    """
    Analyzes the latest U.S. ending stocks (inventories) of crude oil.
    The query is ignored.
    """
    # TODO: Harden this tool
    return "Mock Data: U.S. Crude Oil stocks are at 825,193 (Thousand Barrels)."

# --- 3. Define Your Agents ---

market_analyst = Agent(
  role='Downstream Market Analyst',
  goal='Identify prized markets by analyzing regional demand and pricing.',
  backstory='You are an expert in O&G downstream markets. Your '
            'job is to find which markets are most profitable.',
  verbose=True,
  allow_delegation=False,
  tools=[market_demand_tool],
  llm=gemini_llm
)

operations_analyst = Agent(
  role='Midstream Operations Analyst',
  goal='Analyze the physical operations of the midstream sector, '
       'including refining, logistics, and storage.',
  backstory='You are a data engineer who monitors the physical '
            'flow and processing of oil and gas. You track refinery '
            'utilization, import/export volumes, and inventory levels.',
  verbose=True,
  allow_delegation=False,
  tools=[
      refining_analyzer_tool,
      logistics_volume_tool,
      storage_inventory_tool
  ],
  llm=gemini_llm
)

financial_analyst = Agent(
  role='Energy Financial Analyst',
  goal='Analyze the financial health of the O&G market, focusing '
       'on price benchmarks and midstream profitability.',
  backstory='You are a financial expert who lives in the data. '
            'You track the Henry Hub spot price and '
            'calculate midstream margins.',
  verbose=True,
  allow_delegation=False,
  tools=[market_benchmark_tool, midstream_margin_tool],
  llm=gemini_llm
)

chief_strategist = Agent(
  role='Chief Midstream Strategist',
  goal='Synthesize all market, operational, and financial data '
       'with competitor news to create a holistic predictive report.',
  backstory='You are the "big picture" thinker. You take reports '
            'from your specialist agents (market, operations, financial) '
            'and compare them to real-world competitor news to '
            'assess the company\'s true position.',
  verbose=True,
  allow_delegation=True,
  tools=[search_tool],
  llm=gemini_llm
)

# --- 4. Define Your Tasks (FIXED with Triple Quotes) ---

task_market_analysis = Task(
  description="""Identify the top 2 most "prized" markets.
      Analyze "Houston" and "New York" using your tools
      and report your findings.""",
  expected_output="""A short report on the demand and pricing
      for Houston and New York.""",
  agent=market_analyst
)

task_operations_analysis = Task(
  description="""Provide a full operations report.
      1. Get the latest refinery utilization.
      2. Get the latest logistics volumes (imports/exports).
      3. Get the latest crude oil storage levels.""",
  expected_output="""A 3-part report summarizing the latest
      refining, logistics, and storage data.""",
  agent=operations_analyst
)

task_financial_analysis = Task(
  description="""Provide a full financial report.
      1. Get the latest Henry Hub Natural Gas spot price.
      2. Get the latest midstream margin (Wellhead vs. Citygate).""",
  expected_output="""A 2-part report summarizing the latest
      market benchmarks and margins.""",
  agent=financial_analyst
)

task_final_report = Task(
  description="""Create a final, predictive report.
      1. Use the analyses from your team (Tasks 1, 2, 3).
      2. Search for the latest news on midstream competitors
      like "Kinder Morgan" and "Enterprise Products Partners".
      3. Synthesize ALL information to assess our company's
      current reality versus our competitors.""",
  expected_output="""A comprehensive 4-paragraph report that
      answers all 6 core goals and provides a
      final predictive outlook.""",
  agent=chief_strategist,
  context=[
      task_market_analysis,
      task_operations_analysis,
      task_financial_analysis
  ]
)

# --- 5. Create and Run the Crew ---

oil_and_gas_crew = Crew(
  agents=[
      market_analyst,
      operations_analyst,
      financial_analyst,
      chief_strategist
  ],
  tasks=[
      task_market_analysis,
      task_operations_analysis,
      task_financial_analysis,
      task_final_report
  ],
  process=Process.sequential,
  verbose=True
)

print("--- Kicking off the O&G Analysis Crew (Powered by Gemini) ---")
result = oil_and_gas_crew.kickoff()

print("\n\n--- Crew Run Complete ---")
print("Final Report:")
print(result)